<a href="https://colab.research.google.com/github/gupta1000/image-analogies/blob/master/img_analogies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS445 Final Project: Image Analogies
#### Michael Kokkines (mgk3) & Kush Gupta (kg3)


Something about image analogies

**Setup**

In [0]:
import cv2
import numpy as np
from scipy import signal

%matplotlib inline
from matplotlib import pyplot as plt

from google.colab.patches import cv2_imshow

In [0]:
!pip install annoy
from annoy import AnnoyIndex

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
A_fname = "a.png"
Ap_fname = "a_p.png"
B_fname = "b.png"

In [0]:
A = cv2.imread(A_fname, cv2.IMREAD_UNCHANGED)
cv2_imshow(A)

In [0]:
A_p = cv2.imread(Ap_fname, cv2.IMREAD_UNCHANGED)
cv2_imshow(A_p)

In [0]:
B = cv2.imread(B_fname, cv2.IMREAD_UNCHANGED)
cv2_imshow(A_p)

**Gaussian Computation**

In [0]:
def gaussian_kernel(sigma, kernel_half_size):
    '''
    Inputs:
        sigma = standard deviation for the gaussian kernel
        kernel_half_size = recommended to be at least 3*sigma
    
    Output:
        Returns a 2D Gaussian kernel matrix
    '''
    window_size = kernel_half_size*2+1
    gaussian_kernel_1d = signal.gaussian(window_size, std=sigma).reshape(window_size, 1)
    gaussian_kernel_2d = np.outer(gaussian_kernel_1d, gaussian_kernel_1d)
    gaussian_kernel_2d /= np.sum(gaussian_kernel_2d)
    return gaussian_kernel_2d

In [0]:
def gaussian_pyr(im, levels):
  images = []
  kernel = gaussian_kernel(4, 8)

  prev_img = signal.convolve2d(
        im, 
        kernel,
        boundary="symm",
        mode="same" 
  )
  images.append(prev_img)
  
  for i in range(levels - 1):
    prev_img = cv2.resize(
      signal.convolve2d(
          prev_img, 
          kernel, 
          boundary="symm", 
          mode="same"
        ),
        None, 
        fx=.5, 
        fy=.5
    )
    images.append(prev_img)
  
  return images


def gaussian_pyr_rgb(im, levels):
  red_images = gaussian_pyr(im[:, :, 0], levels)
  green_images = gaussian_pyr(im[:, :, 1], levels)
  blue_images = gaussian_pyr(im[:, :, 2], levels)

  images = []
  for i in range(len(red_images)):
    img = np.zeros((red_images[i].shape[0], red_images[i].shape[1], 3))
    img[:, :, 0] = red_images[i]
    img[:, :, 1] = green_images[i]
    img[:, :, 2] = blue_images[i]
    images.append(img)
  
  return images[::-1]

In [0]:
def build_gaussians(A, A_p, B, L):
  a = gaussian_pyr_rgb(A, L)
  a_p = gaussian_pyr_rgb(A_p, L)
  b = gaussian_pyr_rgb(B, L)
  return a, a_p, b

**Feature Computation**

In [0]:
num_features = 4

In [0]:
def compute_features(im):
  feature_vec = np.zeros((im.shape[0], im.shape[1], num_features))

  # Add the RGB channels
  feature_vec[:, :, 0:3] = im

  # Now, extract luminance
  lab_im = cv2.cvtColor(im.astype('float32'), cv2.COLOR_BGR2LAB)
  feature_vec[:,  :, 3] = lab_im[:, :, 0]
  
  return feature_vec

In [0]:
def compute_featuresets(a, a_p, b, L):
  a_feat = []
  a_p_feat = []
  b_feat = []
  for l in range(L):
    a_feat.append(compute_features(a[l]))
    a_p_feat.append(compute_features(a_p[l]))
    b_feat.append(compute_features(b[l]))
  return a_feat, a_p_feat, b_feat

**Pixel Matching**

*Approximate Matching*

In [0]:
def plant_forest(im, L):
  forests = []

  for level in range(L):
    # We decided to use Manhattan distance
    ann = AnnoyIndex(num_features, "manhattan")
    
    i = 0
    for r in range(im.shape[0]):
      for c in range(im.shape[1]):
        ann.add_item(i, im[r, c, :])
        i += 1
    
    # Create a forest of ten trees
    ann.build(10)
    forests.append(ann)
  
  return forests

In [0]:
def best_approx_match(forest, b, q):
  return forest.get_item_vector(
      forest.get_nns_by_vector(b[q], 1, search_k=forest.get_n_items())[0]
    )

*Feature Distance*

In [0]:
def normalize_f(f):
  norm = np.zeros(f.shape)
  gk = gaussian_kernel(f.shape[0]/6, int(f.shape[0]/2))
  for i in range(num_features):
    norm[:,:,i] = f[:,:,i] * gk
  return norm

In [0]:
# TODO: make this not th most horrible thing in existance
def concat_feat(fpyr, p, l, fine_bound=2, coarse_bound=1):
  p_y = p[0]
  p_x = p[1]

  if (p_x - fine_bound < 0 or p_x + fine_bound >= fpyr[l].shape[1] or p_y - fine_bound < 0 or p_y + fine_bound >= fpyr[l].shape[0]):
    p_x += fine_bound
    p_y += fine_bound
    lpyr = np.zeros((fpyr[l].shape[0]+fine_bound, fpyr[l].shape[1]+fine_bound, fpyr[l].shape[2]))
    lpyr[fine_bound:fine_bound+fpyr[l].shape[0],fine_bound:fine_bound+fpyr[l].shape[1]] = fpyr[l]

    f = normalize_f(lpyr[p_y-fine_bound:p_y+fine_bound+1, p_x-fine_bound:p_x+fine_bound+1]).flatten()

    f_coarse = np.zeros(((2 * coarse_bound + 1)**2*num_features))
    if (l > 0):
      lpyr_coarse = np.zeros((fpyr[l-1].shape[0]+coarse_bound, fpyr[l-1].shape[1]+coarse_bound, fpyr[l-1].shape[2]))
      lpyr_coarse[coarse_bound:coarse_bound+fpyr[l-1].shape[0],coarse_bound:coarse_bound+fpyr[l-1].shape[1]] = fpyr[l-1]
      f_coarse = normalize_f(lpyr_coarse[int(p_y/2)-coarse_bound:int(p_y/2)+coarse_bound+1,int(p_x/2)-coarse_bound:int(p_x/2)+coarse_bound+1]).flatten()
    return np.concatenate((f, f_coarse), axis=0) 

  lpyr = fpyr[l]

  f = normalize_f(lpyr[p_y-fine_bound:p_y+fine_bound+1, p_x-fine_bound:p_x+fine_bound+1]).flatten()

  f_coarse = np.zeros((2 * coarse_bound + 1)**2*num_features)
  if (l > 0):
    f_coarse = normalize_f(fpyr[l-1][int(p_y/2)-coarse_bound:int(p_y/2)+coarse_bound+1,int(p_x/2)-coarse_bound:int(p_x/2)+coarse_bound+1]).flatten()
  return np.concatenate((f, f_coarse), axis=0) 

In [0]:
def F_l(a_f, a_p_f, p, l):
  return np.concatenate((concat_feat(a_f, p, l, fine_bound=2, coarse_bound=1), concat_feat(a_p_f, p, l, fine_bound=2, coarse_bound=1)), axis=0)

In [0]:
def feature_dist(a_f, a_p_f, b_f, b_p_f, p, q, l, fine_bound=2, coarse_bound=1):
  return np.linalg.norm(F_l(a_f, a_p_f, p, l) - F_l(b_f, b_p_f, p, l), ord=2)

In [0]:
print(feature_dist(a_features, a_p_features, b_features, b_features, (1,1), (100,100), 2))

*Coherence Matching*

In [0]:
def best_coherence_match(a_f, b_f, s, q, l):
  

*Matching Core*

In [0]:
# We typically use 2 ≤ K ≤ 25 for color non-photorealistic filters,
# K = 1 for line art filters, and 0.5 ≤ K ≤ 5 for texture synthesis.

K = 1

In [0]:
def best_match():
  p_app = best_approx_match()
  p_coh = best_coherence_match()
  dist_app = feature_dist()
  dist_coh = feature_dist()
  return p_coh if (dist_coh <= (dist_app * (1 + 2^(lvl - L) * K))) else p_app

In [0]:
def match_all():
  for l in range(L):
    for q_y in range(out_h):
      for q_x in range(out_w):
        p = best_match()
        b_p[l][q_y,q_x] = a_p[l][p]
        s[l][q_y,q_x] = p

**Image Analogies Core**

Unit Testing

In [0]:
# levels
L = 3 

# size
out_w = 640
out_h = 480

In [0]:
a, a_p, b = build_gaussians(A, A_p, B, L)

In [0]:
a_features, a_p_features, b_features = compute_featuresets(a, a_p, b, L)

In [0]:
_ = match_all()

Fully Automated Runs

In [0]:
# create image analogy (size: out_w x out_h) for image B using references A : A_p
# using a gaussian pyramid of the specified number of levels 'L'
def create_image_analogy(A, A_p, B, L, out_w, out_h):

  # step 1: create gaussian pyramids for A, A_p, and B
  _ = build_gaussians()
  # step 2: compute features for A, A_p, and B
  _ = compute_featuresets()
  
  # various init stuff

  # step 3: solve for best matches
  _ = match_all()

  return